In [ ]:
import pandas as pd
import io
from tqdm import tqdm
from time import sleep
import numpy as np
import requests
import math
import json
import linecache
import os

In [2]:
filepath = '../data/NAD_r11.txt' # too large to push onto Git, but is part of the download from here: https://nationaladdressdata.s3.amazonaws.com/NAD_r12_TXT.zip

In [3]:
first_line = None

with open(filepath) as f:
    first_line = f.readline()

print(first_line)

OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,Bulk_Plus4,StN_PreMod,StN_PreDir,StN_PreTyp,StN_PreSep,StreetName,StN_PosTyp,StN_PosDir,StN_PosMod,AddNum_Pre,Add_Number,AddNum_Suf,LandmkPart,LandmkName,Building,Floor,Unit,Room,Addtl_Loc,Milepost,Longitude,Latitude,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired



In [4]:
import warnings
warnings.filterwarnings('ignore')

# Load cleaning function

In [5]:
def clean_df(dc):
    def check_int(val):
        num = None
        try:
            num = str(int(float(val)))
        except Exception as e: print(e)
        return num

    dc = dc.replace('nan', '')
    dc = dc.fillna('')   
    dc = dc[dc['Zip_Code'] != 'nan']
    dc = dc[dc['Zip_Code'] != '']
    dc = dc.apply(lambda x: x.astype(str).str.lower()) # convert everything lower case

    # cleaning zip code
    dc['Zip_Code'] = dc['Zip_Code'].apply(lambda x: check_int(x))

    # removing invalid zip rows
    dc = dc[dc['Zip_Code'].notnull()]
    return dc

In [6]:
def translate_lines(first_line, lines, ids):    
    # Given csv lines for a data frame, save it to a given name given state and county
    df = pd.read_csv(io.StringIO(first_line + lines), low_memory=False)
    df = df[['State', 'County', 'Zip_Code', 'Post_Comm', 'Add_Number', 'StN_PreMod', 'StN_PreDir', 'StN_PreTyp', 'StN_PreSep', 'StreetName', 'StN_PosTyp', 'StN_PosDir', 'StN_PosMod', 'Longitude', 'Latitude']]
    df = df.astype(str)   

    df = clean_df(df)

    # constructing address
    df['Prefix'] = df['Add_Number'] + ' ' + df['StN_PreMod'] + ' '+ df['StN_PreDir'] + ' '+ df['StN_PreTyp'] + ' '+ df['StN_PreSep'] + ' '+ df['StreetName'] + ' '+ df['StN_PosTyp'] + ' '+ df['StN_PosDir'] + ' '+ df['StN_PosMod']
    df['Suffix'] = ','+df['Post_Comm'] + ','+ df['State'] + ',' +df['Zip_Code']
    df['Prefix'] = df['Prefix'].apply(lambda x: " ".join(x.split()))
    df['Suffix'] = df['Suffix'].apply(lambda x: " ".join(x.split()))
    df['Address'] = df['Prefix'] + df['Suffix']    

    # rename the columns
    df.rename(columns={
        'State': 'state', 
        'County': 'county', 
        'Zip_Code': 'zip',
        'Add_Number': 'add_number',
        'StreetName': 'street_name',
        'Longitude': 'longitude',
        'Latitude': 'latitude',
        'Address': 'address',
    }, inplace=True)

    # Remove unecessary columns
    df = df[['state', 'county', 'zip', 'longitude', 'latitude', 'address']]
    df['id'] = df['state'] + '_' + df['county']
    df['id'] = df['id'].apply(lambda x: x.replace(' ', '_'))
    ids|= set(df['id'].unique())
    return df

#     # Save to a compressed csv
#     filename = '../data/%s_%s.csv.xz' % (state, county.replace(' ', '_'))
#     # print('Saving to: ', filename)
#     df.to_csv(filename, index=False)
#     sc[state] = '%s_%s.csv.xz' % (state, county)

# Parse state county until a new one, then save the data frame and go next

In [7]:
dfs = []
ids = set()
steps = 5000
total_lines = 67357690
with tqdm(total=int(total_lines)) as pbar:
    with open(filepath) as f:
        first_line = f.readline() # skip first line
        i = 0
        lines = ''
        for line in f:
            # pbar.set_description('df length: %s' % len(dfs))
            pbar.update(1)
            lines += line
            
            if i > steps:
                df = translate_lines(first_line, lines, ids)
                dfs.append(df)
                i =0
                lines = ''
            i += 1
pbar.close()      
# Capture the leftovers
remain_df = pd.read_csv(io.StringIO(first_line + lines), low_memory=False)
dfs.append(remain_df)

  5%|█▍                           | 3420686/67357690 [00:37<11:57, 89131.66it/s]

could not convert string to float: 'ia'


  6%|█▊                           | 4180838/67357690 [00:46<11:48, 89223.89it/s]

could not convert string to float: '5-156'


 17%|████▊                       | 11627327/67357690 [02:06<09:59, 92919.04it/s]

...

 98%|███████████████████████████▍| 65888177/67357690 [12:00<00:14, 98875.01it/s]

could not convert string to float: 'acme'
could not convert string to float: 'acme'
could not convert string to float: 'acme'
could not convert string to float: 'acme'


100%|███████████████████████████▉| 67088417/67357690 [12:12<00:02, 92483.46it/s]

could not convert string to float: 'out'
could not convert string to float: 'out'


100%|███████████████████████████▉| 67357689/67357690 [12:15<00:00, 91610.99it/s]


In [8]:
print(len(dfs))

13469


In [9]:
dfs[0]

,state,county,zip,longitude,latitude,address,id
0,az,coconino,86336,-111.771770637,34.8280627940001,"95 lynx drive,sedona,az,86336",az_coconino
1,az,coconino,86336,-111.769395183,34.828455976,"33 eagle lane,sedona,az,86336",az_coconino
2,az,coconino,86336,-111.777281707,34.828623958,"141 vista bonita drive,sedona,az,86336",az_coconino
3,az,coconino,86336,-111.775765885,34.8291778600001,"109 badger drive,sedona,az,86336",az_coconino
4,az,coconino,86336,-111.773383133,34.824991696,"30 fawn spur,sedona,az,86336",az_coconino
...,...,...,...,...,...,...,...
4997,az,gila,85541,-111.303339925,34.3157243200001,"297 west patriot drive,payson,az,85541",az_gila
4998,az,gila,85541,-111.302823657,34.3163382070001,"282 west patriot drive,payson,az,85541",az_gila
4999,az,gila,85541,-111.218377852,34.2650574410001,"139 north pinon road,star valley,az,85541",az_gila
5000,az,gila,85554,-110.98229521,34.107269951,"311 north seeley road,young,az,85554",az_gila


In [15]:
dfs[0].columns

Index(['state', 'county', 'zip', 'longitude', 'latitude', 'address', 'id'], dtype='object')

In [16]:
pbar = tqdm(ids)

for uid in pbar:
    udf = []
    for df in dfs:
        if not 'id' in df.columns:
            continue 
        # assuming 'id' in the data frame
        cut = df[df['id'] == uid]
        if not cut.empty:
            udf.append(cut)

    pbar.set_description('Parsing %s' % uid)
    if len(udf) > 0:
        pbar.set_description('Saving %s' % uid)        
        udf = pd.concat(udf)
        udf.to_csv('../data/%s.csv.xz' % uid, index=False)

Saving tn_putnam: 100%|███████████████████| 1585/1585 [3:28:19<00:00,  7.89s/it]


In [13]:
# check that there are no spaces in the identifier
for uid in ids:
    if ' ' in uid:
        print(uid)

In [27]:
len(ids)

1585

In [28]:
with open('../data/state_county.json','w') as f:
    json.dump(list(ids), f)

print(os.path.isfile('../data/state_county.json'))

True


# Sample verify address

In [17]:
# url = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
# s_buf = io.StringIO()
# # saving a data frame to a buffer (same as with a regular file):
# # s = '\n'.join(list(df['Address']))
# # print(s)
# s_buf = io.StringIO()
# # saving a data frame to a buffer (same as with a regular file):
# df['Address'].to_csv(s_buf, index=False)
# df['Address'].to_csv('../data/temp_addr.csv', index=False)
# print(df['Address'])

In [18]:
# test_address = df['address'].sample(1).values[0]
# print(test_address)
# r = requests.get('https://geocoding.geo.census.gov/geocoder/locations/onelineaddress?address=%s&benchmark=2020&format=json' % test_address).json()
# print(r)  
# print(len(r['result']['addressMatches']) > 0)

In [19]:
# # https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
# # curl --form addressFile=@localfile.csv --form benchmark=2020 https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output geocoderesult.csv
# # https://curlconverter.com/

# url = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
# files = {
#     'addressFile': open('../data/temp_addr.csv', 'r'),
#     'benchmark': (None, '2020'),
# }

# response = requests.post('https://geocoding.geo.census.gov/geocoder/locations/addressbatch', files=files)
# print(response.content)
# print(response)

In [20]:
# len(df['State'].unique()) # check number of different states